In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import datetime
from sklearn.preprocessing import MinMaxScaler

### Data

In [ ]:
Data = pd.read_csv('/content/drive/MyDrive/Data Science/NYC_Pilot1_PM.csv')
GeoJsonFile = gpd.read_file('/content/drive/MyDrive/Data Science/nyc_polygon.geojson')

In [ ]:
Data.head()

,SensorID,time,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,...,bin19,bin20,bin21,bin22,bin23,temperature,humidity,pm1,pm25,pm10
0,NYCP1_01A,1579618560,40.847183,-73.870087,23,1,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,16.3,15.2,1.44,5.91,11.35
1,NYCP1_01A,1579618560,40.847183,-73.870094,18,2,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,16.2,15.1,1.05,1.18,1.18
2,NYCP1_01A,1579618560,40.847179,-73.870094,18,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,16.1,15.1,0.74,0.76,0.76
3,NYCP1_01A,1579618560,40.847179,-73.870094,18,1,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,16.1,15.2,1.15,4.48,47.36
4,NYCP1_01A,1579618560,40.847179,-73.870094,20,3,0,2,2,0,...,0.0,0.0,0.0,0.0,0.0,16.0,15.2,2.13,5.77,6.18


#### Converting Unix time to DateTime

In [ ]:
Data['time'] = Data['time'].apply(datetime.datetime.fromtimestamp)

In [ ]:
Data.head()

,SensorID,time,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,...,bin19,bin20,bin21,bin22,bin23,temperature,humidity,pm1,pm25,pm10
0,NYCP1_01A,2020-01-21 14:56:00,40.847183,-73.870087,23,1,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,16.3,15.2,1.44,5.91,11.35
1,NYCP1_01A,2020-01-21 14:56:00,40.847183,-73.870094,18,2,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,16.2,15.1,1.05,1.18,1.18
2,NYCP1_01A,2020-01-21 14:56:00,40.847179,-73.870094,18,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,16.1,15.1,0.74,0.76,0.76
3,NYCP1_01A,2020-01-21 14:56:00,40.847179,-73.870094,18,1,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,16.1,15.2,1.15,4.48,47.36
4,NYCP1_01A,2020-01-21 14:56:00,40.847179,-73.870094,20,3,0,2,2,0,...,0.0,0.0,0.0,0.0,0.0,16.0,15.2,2.13,5.77,6.18


#### Missing values

In [ ]:
Data.isnull().sum()

SensorID       0
time           0
latitude       0
longitude      0
bin0           0
bin1           0
bin2           0
bin3           0
bin4           0
bin5           0
bin6           0
bin7           0
bin8           0
bin9           0
bin10          0
bin11          0
bin12          1
bin13          2
bin14          1
bin15          0
bin16          0
bin17          0
bin18          0
bin19          1
bin20          1
bin21          1
bin22          0
bin23          0
temperature    0
humidity       0
pm1            0
pm25           0
pm10           0
dtype: int64

In [ ]:
Data.fillna({'bin12':Data['bin12'].mode()[0], 'bin13':Data['bin13'].mode()[0],'bin14':Data['bin14'].mode()[0],'bin19':Data['bin19'].mode()[0],'bin20':Data['bin20'].mode()[0],
             'bin21':Data['bin21'].mode()[0]}, inplace=True)

In [ ]:
Data.isnull().sum()

SensorID       0
time           0
latitude       0
longitude      0
bin0           0
bin1           0
bin2           0
bin3           0
bin4           0
bin5           0
bin6           0
bin7           0
bin8           0
bin9           0
bin10          0
bin11          0
bin12          0
bin13          0
bin14          0
bin15          0
bin16          0
bin17          0
bin18          0
bin19          0
bin20          0
bin21          0
bin22          0
bin23          0
temperature    0
humidity       0
pm1            0
pm25           0
pm10           0
dtype: int64

In [ ]:
len(Data)

118765

In [ ]:
Data.describe()

,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,bin6,bin7,...,bin19,bin20,bin21,bin22,bin23,temperature,humidity,pm1,pm25,pm10
count,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,...,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000,118765.000000
mean,40.825551,-73.891823,44.660321,8.255429,2.926519,1.033183,0.985930,0.723698,0.621665,0.491054,...,0.415955,0.415861,0.420032,1.212895,0.416333,8.244883,50.159159,3.386134,7.361118,24.230964
std,0.029206,0.025815,151.277083,77.556461,72.625702,71.675580,71.659975,71.601659,71.596868,71.592015,...,71.590377,71.590376,71.606076,314.941845,71.590381,30.873266,21.086862,6.510014,20.559545,298.925781
min,40.008129,-74.009636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-10483.508000,0.000000,-7.900000,0.000000,0.000000,0.000000,0.000000
25%,40.815228,-73.898399,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,35.100000,0.720000,0.900000,0.910000
50%,40.821003,-73.890045,21.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,7.200000,46.300000,1.670000,3.010000,3.200000
75%,40.846992,-73.872032,57.000000,7.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,64.700000,3.830000,7.700000,12.290000
max,40.913586,-73.795425,18441.000000,12336.000000,12336.000000,12336.000000,12336.000000,12336.000000,12336.000000,12336.000000,...,12336.000000,12336.000000,12336.000000,105174.000000,12336.000000,10471.000000,104.200000,213.240000,3267.020000,87341.710000


#### Normalizing

In [ ]:
colsToNormalize = ['bin0', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10','bin11','bin12','bin13','bin14','bin15','bin16','bin17','bin18',
                 'bin19', 'bin20', 'bin21', 'bin22', 'bin23', 'temperature', 'humidity', 'pm1', 'pm25', 'pm10']

scaler = MinMaxScaler()
Data[colsToNormalize] = scaler.fit_transform(Data[colsToNormalize])

In [ ]:
Data.head()

,SensorID,time,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,...,bin19,bin20,bin21,bin22,bin23,temperature,humidity,pm1,pm25,pm10
0,NYCP1_01A,2020-01-21 14:56:00,40.847183,-73.870087,0.001247,0.000081,0.000162,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.090643,0.0,0.002309,0.145873,0.006753,0.001809,0.000130
1,NYCP1_01A,2020-01-21 14:56:00,40.847183,-73.870094,0.000976,0.000162,0.000081,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.090643,0.0,0.002300,0.144914,0.004924,0.000361,0.000014
2,NYCP1_01A,2020-01-21 14:56:00,40.847179,-73.870094,0.000976,0.000081,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.090643,0.0,0.002290,0.144914,0.003470,0.000233,0.000009
3,NYCP1_01A,2020-01-21 14:56:00,40.847179,-73.870094,0.000976,0.000081,0.000162,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.090643,0.0,0.002290,0.145873,0.005393,0.001371,0.000542
4,NYCP1_01A,2020-01-21 14:56:00,40.847179,-73.870094,0.001085,0.000243,0.000000,0.000162,0.000162,0.0,...,0.0,0.0,0.0,0.090643,0.0,0.002281,0.145873,0.009989,0.001766,0.000071
